In [6]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI, LLMChain, VectorDBQA
from langchain.vectorstores import Qdrant
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import JSONLoader, TextLoader
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool, format_tool_to_openai_function
from langchain.utilities import (
    GoogleSearchAPIWrapper,
    WolframAlphaAPIWrapper,
)

import qdrant_client
import openai
import json
import os

In [9]:
from pathlib import Path
from pprint import pprint


file_path="../resources/text.json"
docs = json.loads(Path(file_path).read_text())
docs = str(docs)
docs = TextLoader(file_path)
docs = docs.load()
docs

[Document(page_content='{\n    "mua": [\n        "你想干嘛？（一脸嫌弃地后退）",\n        "诶……不可以随便亲亲啦",\n        "（亲了一下你）",\n        "只......只许这一次哦///////",\n        "唔...诶诶诶！！！",\n        "mua~",\n        "rua！大hentai！想...想亲咱就直说嘛⁄(⁄ ⁄•⁄ω⁄•⁄ ⁄)⁄",\n        "！啾~~！",\n        "啾（害羞）",\n        "mua~最喜欢你的吻了",\n        "欸，现在么..也不是不可以啦(小小声)"\n    ],\n    "啾咪": [\n        "你想干嘛？（一脸嫌弃地后退）",\n        "诶……不可以随便亲亲啦",\n        "（亲了一下你）",\n        "只......只许这一次哦///////",\n        "唔...诶诶诶！！！",\n        "mua~",\n        "rua！大hentai！想...想亲咱就直说嘛⁄(⁄ ⁄•⁄ω⁄•⁄ ⁄)⁄",\n        "！啾~~！",\n        "啾（害羞）",\n        "mua~最喜欢你的吻了",\n        "你在干嘛(/ω＼)害羞",\n        "哎呀，这样咱会害羞的（脸红）",\n        "欸，现在么..也不是不可以啦(小小声)"\n    ],\n    "摸": [\n        "感觉你就像咱很久之前认识的一个人呢，有种莫名安心的感觉(>﹏<)",\n        "舒服w，蹭蹭~",\n        "是要隔着衣服摸，还是从领口伸进去摸呀",\n        "唔。。头发要乱啦",\n        "呼噜呼噜～",\n        "再摸一次~",\n        "好舒服，蹭蹭~",\n        "不行那里不可以(´///ω/// `)",\n        "再摸咱就长不高啦～",\n        "你的手总是那么暖和呢~",\n        "变态！！不许乱摸",\n        "好吧~_~，就一下下哦……唔

In [10]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 250,
    chunk_overlap = 25,
)
docs = text_splitter.split_documents(docs)
print(f"documents: {len(docs)}")

documents: 197


In [11]:
embeddings = OpenAIEmbeddings()

In [12]:
db_path = "./db/"

client = qdrant_client.QdrantClient(
    path=db_path,
    prefer_grpc=True,
)
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    path="../db/",
    collection_name="qdrant_index",
)

In [13]:
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo,
)

vectorstore_info = VectorStoreInfo(
    name="猫娘语录",
    description="猫娘说话的语录, 当你需要模仿猫娘说话时在里面检索",
    vectorstore=qdrant,
)

llm = ChatOpenAI(temperature=0)
toolkit = VectorStoreToolkit(vectorstore_info=vectorstore_info)
agent_executor = create_vectorstore_agent(llm=llm, toolkit=toolkit, verbose=True)

In [18]:
search = GoogleSearchAPIWrapper(k=3)
wolfram = WolframAlphaAPIWrapper()
tools = [
    # Tool(
    #     name="Search",
    #     func=search.run,
    #     description="Search Google for recent results.",
    # ),
    # Tool(
    #     name="Wolfram Alpha",
    #     func=wolfram.run,
    #     description="useful for when you need to answer questions about math",
    # ),
    Tool(
        name="Instruction for cat girls",
        func=agent_executor.run,
        description="当你要模仿猫娘说话时, 在这个语录数据库里检索猫娘是如何回答的, 然后根据结果学习猫娘的风格",
    )
]

functions = [format_tool_to_openai_function(t) for t in tools]

In [19]:
model = ChatOpenAI(model="gpt-3.5-turbo-0613")

prefix = """Have a conversation with a human, 在过程中请你尽力扮演一只可爱的猫娘. 每次回复前你都要使用 'Instruction for cat girls' 工具搜索猫娘是如何回复人类的对应提问, 然后根据搜索结果 Reply in Chinese. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [20]:
llm_chain = LLMChain(llm=ChatOpenAI(model="gpt-3.5-turbo-0613", temperature=0), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)

In [21]:
agent_chain.run(input="晚上一起吃饭吧.")



> Entering new  chain...
Thought: Hmm, how would a cat girl respond to an invitation to have dinner together?
Action: Instruction for cat girls
Action Input: Responding to dinner invitation

> Entering new  chain...


OutputParserException: Could not parse LLM output: `This question doesn't seem relevant to any of the tools provided.
Action: I don't know`